In [44]:
import pandas as pd
import numpy as np
import datetime
from scorepi import *
from epiweeks import Week
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import matplotlib as mpl
import random
from datetime import timedelta
from numba import njit

import warnings
warnings.filterwarnings('ignore')

# Energy Score

In [2]:
# calculate the energy score using this function
# X is matrix of trajectories, y is observations. Both must be numpy arrays.

@njit
def energyscore(X,y):
    ES = 0
    N = X.shape[0]
    for i in range(N):
        ES += np.sqrt(np.sum((X[i]-y)**2))/N
    for i in range(N):
        for j in range(N):
            ES -= np.sqrt(np.sum((X[i]-X[j])**2))/(2*N**2)
    return ES


In [19]:
# example calculation of energy score
                
N = 100 # number of trajectories
M = 20 # number of time points

# generate trajectories that are drawn from a standard normal distribution at each time point
X = np.array([np.random.normal(size=M) for _ in range(N)]) 

# generate ground truth data where the first time point is drawn from a standard normal and all future time points
# equal the first time point
y = np.random.normal()*np.ones(M)


ES = energyscore(X,y)


# Normalized Energy Score

In [14]:
# calculate the normalized energy score using this function
# X is matrix of trajectories, y is observations. Both must be numpy arrays.

@njit
def normalized_energyscore(X,y):
    ES = 0
    N = X.shape[0]
    for i in range(N):
        ES += np.sqrt(np.sum(((X[i]-y)/y)**2))/N
    for i in range(N):
        for j in range(N):
            ES -= np.sqrt(np.sum(((X[i]-X[j])/y)**2))/(2*N**2)
    return ES

In [21]:
# example calculation of normalized energy score
                
N = 100 # number of trajectories
M = 20 # number of time points

# generate trajectories that are drawn from a standard normal distribution at each time point
X = np.array([np.random.normal(size=M) for _ in range(N)]) 

# generate ground truth data where the first time point is drawn from a standard normal and all future time points
# equal the first time point
y = np.random.normal()*np.ones(M)


ESnorm = normalized_energyscore(X,y)


# Sampled Energy Score

In [34]:
N = 10 # number of total trajectories
M = 20 # number of time points
num_samples = 10 # number of trajectories to sample from full group of trajectories

# generate trajectories that are drawn from a standard normal distribution at each time point
X = np.array([np.random.normal(size=M) for _ in range(N)]) 

# generate ground truth data where the first time point is drawn from a standard normal and all future time points
# equal the first time point
y = np.random.normal()*np.ones(M)


Xsampled = np.array(random.choices(X, k=num_samples)) # get group of sampled trajectories
ES_sampled = energyscore(Xsampled,y)


# Using Flu Scenario Modeling Hub data to calculate the Energy Score

In [38]:
# pull surveillance data and model predictions from Flu Scenario Modeling Hub GitHub
# https://github.com/midas-network/flu-scenario-modeling-hub/tree/main

def pull_surveillance_data():
    mapping = {'death':'Deaths', 'case':'Cases', 'hospitalization': 'Hospitalizations'}
    
    url = f"https://raw.githubusercontent.com/cdcepi/FluSight-forecast-hub/main/target-data/target-hospital-admissions.csv"
    return pd.read_csv(url, dtype={'location':str})



def pull_flu_scenario_modeling_hub_predictions(model,dates):
    """pull_scenario_modeling_hub_predictions. Load predictions of the model saved by the scenario modeling
    hub.

    Parameters  
    ----------
    model : str
        Model name on thhe
    dates : list or string
        List of potential dates in the iso format, e.g., 'yyyy-mm-dd', for the submission.
    """
    predictions = None
    if isinstance(dates,str):
        dates = [dates]
    for date in dates:
        url = f"https://raw.githubusercontent.com/midas-network/flu-scenario-modeling-hub/master/data-processed/{model}/{date}-{model}"
        for ext in [".gz.parquet", ".parquet"]:
                try:
                    predictions = pd.read_parquet(url+ext)
                    
                except:
                    pass

    if predictions is None:
        print(f"Data for model {model} and date {dates} unavailable")
    return predictions


In [56]:
# get surveillance data

observations = pull_surveillance_data()
observations['date'] = pd.to_datetime(observations['date'])

#filter start - end week
start_week = pd.to_datetime('2023-09-03')
max_date = pd.to_datetime(observations.date.max())
observations = observations[(observations['date'] >= start_week) & \
                            (observations['date'] <= max_date)].drop(columns=['Unnamed: 0', 'weekly_rate'])

#aggregate to weekly
observations = observations.groupby(['location', pd.Grouper(key='date', freq='W-SAT')]).sum().reset_index()

#transform to Observation object
observations = Observations(observations)


In [57]:
observationse

,location,date,value
0,01,2023-09-09,5
1,45,2023-09-09,22
2,16,2023-09-09,2
3,46,2023-09-09,3
4,15,2023-09-09,5
...,...,...,...
1797,72,2024-04-27,44
1798,35,2024-04-27,20
1799,19,2024-04-27,22
1800,55,2024-04-27,38


In [60]:
# get all individual model predictions

models = [  'Ensemble', 'Ensemble_LOP', 'Ensemble_LOP_untrimmed',
          'MOBS_NEU-GLEAM_FLU',  'NIH-Flu_TS', 'NotreDame-FRED', 'PSI-M2', 'USC-SIkJalpha', 'UT-ImmunoSEIRS']
dates = '2023-09-03'
rd = 4

predictionsall = pd.DataFrame()
for model in models:
    predictions = pull_flu_scenario_modeling_hub_predictions(model,dates) #pull predictions from GitHub
    predictions['Model'] = model
    
    predictions = predictions[predictions['target'] =='inc hosp']

    numweeks = list(predictions.horizon.unique())

    start_date = list(predictions.origin_date.unique())[0]
    date_1 = pd.to_datetime(start_date)

    alldates = []
    for wk in numweeks:
        if wk==1:
            d = date_1 + timedelta(days=6*int(wk))
        else:
            d = pd.to_datetime("2023-09-02") + timedelta(weeks=int(wk))

        alldates.append(d)

    dfdates = pd.DataFrame({'horizon':numweeks, 'target_end_date':alldates}) # add date of prediction to dataframe
    df = predictions.merge(dfdates, how='inner', on='horizon')

    
    predictionsall = pd.concat([predictionsall, df])

In [61]:
# filter by trajectories and only look at age group with all ages combined
predictions_traj = predictionsall[(predictionsall.output_type == 'sample') & \
                                   (predictionsall.age_group == '0-130')]
# filter by dates with data
predictions_traj = predictions_traj[predictions_traj.target_end_date <= pd.to_datetime(observations.date.max())]

Calculate energy score for individual models at each scenario, location, and week

In [62]:
target = 'inc hosp'

energyscoresdf = pd.DataFrame()
for model in predictions_traj.Model.unique():
    for loc in predictions_traj.location.unique():
        
        if loc in ['60','66','69', '72', '78']: #filter out territories
            continue
        
        for scenario in ['A', 'B', 'C', 'D', 'E', 'F']:

            # filter predictions by model, location, and scenario
            predictionsfilt = predictions_traj[(predictions_traj.scenario_id == scenario + '-2023-08-14') & \
                                        (predictions_traj.location == loc) & \
                                        (predictions_traj.Model == model) & \
                                        (predictions_traj.target_end_date <= max_date) & \
                                        (predictions_traj.target_end_date >= start_week)]

            if len(predictionsfilt) == 0:
                print(f'no predictions for {model} at location {loc} for scenario {scenario}')
                continue
        
            #filter location
            obs = observations[observations['location'] == loc]
            
            #calculate energy score

            y = np.array(obs.value)
            X = [np.array(predictionsfilt[predictionsfilt['output_type_id'] == i].value) for i in predictionsfilt['output_type_id'].unique()]

            ES = energyscore(np.array(X),y) #calculate energy score
        
        
            # save scores
            
            if loc == 'US':
                loc_conv = loc
            elif int(loc) <10:
                loc_conv = loc[1]
            else:
                loc_conv = loc  

            newrow = pd.DataFrame({'Model':model,'Label': 'Scenario '+ scenario, 'location':loc_conv, 'energyscore':ES, 
                                'target':target}, index=[0])

            energyscoresdf = pd.concat([energyscoresdf, newrow]) #store all scores in one dataframe

energyscoresdf = energyscoresdf.reset_index()
energyscoresdf = energyscoresdf.drop(columns=['index'])   


no predictions for NIH-Flu_TS at location 01 for scenario A
no predictions for NIH-Flu_TS at location 01 for scenario B
no predictions for NIH-Flu_TS at location 01 for scenario C
no predictions for NIH-Flu_TS at location 01 for scenario D
no predictions for NIH-Flu_TS at location 01 for scenario E
no predictions for NIH-Flu_TS at location 01 for scenario F
no predictions for NIH-Flu_TS at location 02 for scenario A
no predictions for NIH-Flu_TS at location 02 for scenario B
no predictions for NIH-Flu_TS at location 02 for scenario C
no predictions for NIH-Flu_TS at location 02 for scenario D
no predictions for NIH-Flu_TS at location 02 for scenario E
no predictions for NIH-Flu_TS at location 02 for scenario F
no predictions for NIH-Flu_TS at location 04 for scenario A
no predictions for NIH-Flu_TS at location 04 for scenario B
no predictions for NIH-Flu_TS at location 04 for scenario C
no predictions for NIH-Flu_TS at location 04 for scenario D
no predictions for NIH-Flu_TS at locatio

no predictions for NIH-Flu_TS at location 33 for scenario F
no predictions for NIH-Flu_TS at location 34 for scenario A
no predictions for NIH-Flu_TS at location 34 for scenario B
no predictions for NIH-Flu_TS at location 34 for scenario C
no predictions for NIH-Flu_TS at location 34 for scenario D
no predictions for NIH-Flu_TS at location 34 for scenario E
no predictions for NIH-Flu_TS at location 34 for scenario F
no predictions for NIH-Flu_TS at location 37 for scenario A
no predictions for NIH-Flu_TS at location 37 for scenario B
no predictions for NIH-Flu_TS at location 37 for scenario C
no predictions for NIH-Flu_TS at location 37 for scenario D
no predictions for NIH-Flu_TS at location 37 for scenario E
no predictions for NIH-Flu_TS at location 37 for scenario F
no predictions for NIH-Flu_TS at location 38 for scenario A
no predictions for NIH-Flu_TS at location 38 for scenario B
no predictions for NIH-Flu_TS at location 38 for scenario C
no predictions for NIH-Flu_TS at locatio

In [63]:
energyscoresdf

,Model,Label,location,energyscore,target
0,MOBS_NEU-GLEAM_FLU,Scenario A,1,426.430823,inc hosp
1,MOBS_NEU-GLEAM_FLU,Scenario B,1,410.700957,inc hosp
2,MOBS_NEU-GLEAM_FLU,Scenario C,1,410.799822,inc hosp
3,MOBS_NEU-GLEAM_FLU,Scenario D,1,408.848089,inc hosp
4,MOBS_NEU-GLEAM_FLU,Scenario E,1,421.623785,inc hosp
...,...,...,...,...,...
1627,UT-ImmunoSEIRS,Scenario B,US,25598.171912,inc hosp
1628,UT-ImmunoSEIRS,Scenario C,US,27081.777290,inc hosp
1629,UT-ImmunoSEIRS,Scenario D,US,25498.213156,inc hosp
1630,UT-ImmunoSEIRS,Scenario E,US,24345.522962,inc hosp


# Using Flu Scenario Modeling Hub data to calculate the WIS from reported quantiles

In [66]:
# get average WIS over all projection weeks for each model, scenario and location calculated using 
# reported quantiles

# use scorepi package to calculate WIS (https://github.com/gstonge/scorepi/tree/main)


# filter by quantiles and only look at age group with all ages combined
predictions_quant = predictionsall[(predictionsall.output_type == 'quantile') & \
                                   (predictionsall.age_group == '0-130')]

# filter by dates with data
predictions_quant = predictions_quant[predictions_quant.target_end_date <= pd.to_datetime(observations.date.max())]
predictions_quant['target_end_date'] = pd.to_datetime(predictions_quant['target_end_date'])


start_week = pd.to_datetime('2023-09-09')
max_date = pd.to_datetime(observations.date.max())

wisdf = pd.DataFrame()

for model in predictions_quant.Model.unique():
    for loc in predictions_traj.location.unique():
        
        if loc in ['60','66','69', '72', '78']:
            continue
        
        for scenario in ['A', 'B', 'C', 'D', 'E', 'F']:
            
            target = 'hosp'
            
            # filter predictions
            predictionsfilt = predictions_quant[(predictions_quant.scenario_id == scenario + '-2023-08-14') & \
                                        (predictions_quant.location == loc) & \
                                        (predictions_quant.Model == model) & \
                                        (predictions_quant.target_end_date <= max_date) & \
                                        (predictions_quant.target_end_date >= start_week)]
            
            predictionsfilt = predictionsfilt.rename(columns={'output_type_id':'quantile', 'output_type':'type'})
            predictionsfilt['quantile'] = predictionsfilt['quantile'].astype("float")

            if len(predictionsfilt) == 0:
                print(f'no predictions for {model} at location {loc} for scenario {scenario}')
                continue
        
            #filter location
            obs = observations[observations['location'] == loc]

            obs = obs[['location', 'date', 'value']]
            
            #transform to Observation object
            obs = Observations(obs)

            ## calculate WIS 
            
            scenarios = list(predictionsfilt['scenario_id'].drop_duplicates())
            predictions_list = [Predictions(predictionsfilt[predictionsfilt['scenario_id'] == scenario],
                                            t_col='target_end_date') for scenario in scenarios]

            labels = ["Scenario " + scenario[0] for scenario in sorted(scenarios)]

            aggregated_scores = dict()

            # calculate WIS
            for label,predictions in zip(labels,predictions_list):
                d,_ = score_utils.all_scores_from_df(obs, predictions, mismatched_allowed=False)
                
                # to get score for each individual week, use score_utils.all_timestamped_scores_from_df function

                aggregated_scores[label] = d

            
            for label in labels:
                wis = aggregated_scores[label]['wis_mean']
                

            # format output 
            
            if loc == 'US':
                loc_conv = loc
            elif int(loc) <10:
                loc_conv = loc[1]
            else:
                loc_conv = loc  

            newrow = pd.DataFrame({'Model':model,'Label': 'Scenario '+ scenario, 'location':loc_conv, 'WIS':wis, 
                                'target':target}, index=[0])

            wisdf = pd.concat([wisdf, newrow])

wisdf = wisdf.reset_index()
wisdf = wisdf.drop(columns=['index'])   

no predictions for NIH-Flu_TS at location 01 for scenario A
no predictions for NIH-Flu_TS at location 01 for scenario B
no predictions for NIH-Flu_TS at location 01 for scenario C
no predictions for NIH-Flu_TS at location 01 for scenario D
no predictions for NIH-Flu_TS at location 01 for scenario E
no predictions for NIH-Flu_TS at location 01 for scenario F
no predictions for NIH-Flu_TS at location 02 for scenario A
no predictions for NIH-Flu_TS at location 02 for scenario B
no predictions for NIH-Flu_TS at location 02 for scenario C
no predictions for NIH-Flu_TS at location 02 for scenario D
no predictions for NIH-Flu_TS at location 02 for scenario E
no predictions for NIH-Flu_TS at location 02 for scenario F
no predictions for NIH-Flu_TS at location 04 for scenario A
no predictions for NIH-Flu_TS at location 04 for scenario B
no predictions for NIH-Flu_TS at location 04 for scenario C
no predictions for NIH-Flu_TS at location 04 for scenario D
no predictions for NIH-Flu_TS at locatio

no predictions for NIH-Flu_TS at location 34 for scenario A
no predictions for NIH-Flu_TS at location 34 for scenario B
no predictions for NIH-Flu_TS at location 34 for scenario C
no predictions for NIH-Flu_TS at location 34 for scenario D
no predictions for NIH-Flu_TS at location 34 for scenario E
no predictions for NIH-Flu_TS at location 34 for scenario F
no predictions for NIH-Flu_TS at location 37 for scenario A
no predictions for NIH-Flu_TS at location 37 for scenario B
no predictions for NIH-Flu_TS at location 37 for scenario C
no predictions for NIH-Flu_TS at location 37 for scenario D
no predictions for NIH-Flu_TS at location 37 for scenario E
no predictions for NIH-Flu_TS at location 37 for scenario F
no predictions for NIH-Flu_TS at location 38 for scenario A
no predictions for NIH-Flu_TS at location 38 for scenario B
no predictions for NIH-Flu_TS at location 38 for scenario C
no predictions for NIH-Flu_TS at location 38 for scenario D
no predictions for NIH-Flu_TS at locatio

In [67]:
wisdf

,Model,Label,location,WIS,target
0,Ensemble,Scenario A,1,32.096365,hosp
1,Ensemble,Scenario B,1,23.844404,hosp
2,Ensemble,Scenario C,1,32.012242,hosp
3,Ensemble,Scenario D,1,21.510710,hosp
4,Ensemble,Scenario E,1,31.561645,hosp
...,...,...,...,...,...
1633,USC-SIkJalpha,Scenario B,US,3186.455519,hosp
1634,USC-SIkJalpha,Scenario C,US,3184.368746,hosp
1635,USC-SIkJalpha,Scenario D,US,3186.795785,hosp
1636,USC-SIkJalpha,Scenario E,US,3191.250160,hosp


# Using Flu Scenario Modeling Hub data to calculate the WIS by extracting quantiles from trajectories

In [82]:
# calculate WIS by finding quantiles from reported trajectories (not all models submit quantiles)

modelsall = ['MOBS_NEU-GLEAM_FLU', 'USC-SIkJalpha', 'UT-ImmunoSEIRS', 'UVA-FluXSim']

wisdf_traj = pd.DataFrame()

predictions = pd.DataFrame()
i=0
for model in modelsall:
    df = predictionsall[predictionsall.Model == model]
    # filter by trajectories and only look at age group with all ages combined
    df = df[(df.output_type == 'sample') & (df.age_group == '0-130')]
    df['output_type_id'] = df['output_type_id'].astype("int")
    
    df['Model'] = model
    df['trajectory_id'] = df['output_type_id'] + 100*i
    predictions = pd.concat([predictions, df])
    i += 1
    
for model in modelsall:

    for loc in predictions.location.unique():
    
        if loc in ['60','66','69', '72', '78']:
            continue
            
        for scenario in ['A', 'B', 'C', 'D', 'E', 'F']:
            #scenario = 'B'
            location = loc
            target = 'hosp'
            incidence = True
            
            #filter predictions to find WIS for each model, scenario, and location
            predictionsfilt = predictions[(predictions.scenario_id == scenario + '-2023-08-14') & \
                                        (predictions.location == location) & \
                                        (predictions.target == 'inc ' + target)  & \
                                        (predictions.target_end_date <= max_date) & \
                                        (predictions.target_end_date >= start_week) &\
                                        (predictions.Model == model)]

            if len(predictionsfilt)==0:
                continue
            
            obs = observations[observations.location==loc]

            y = np.array(obs.value)
            X = np.array([np.array(predictionsfilt[predictionsfilt.trajectory_id == i].value) \
                            for i in predictionsfilt.trajectory_id.unique()])

            quantiles=[0.01,0.025,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.60,
                        0.65,0.7,0.75,0.8,0.85,0.9,0.95,0.975,0.99]

            # get quantiles
            Q = np.quantile(X,quantiles,axis=0)

            # calculate WIS
            WIS = np.zeros(X.shape[1])
            for i in range(len(quantiles) // 2):
                interval_range = 100*(quantiles[-i-1]-quantiles[i])
                alpha = 1-(quantiles[-i-1]-quantiles[i])
                IS = interval_score(y,Q[i],Q[-i-1],interval_range)
                WIS += IS['interval_score']*alpha/2
            WIS += 0.5*np.abs(Q[len(quantiles) // 2 +1] - y)

            WIS = np.mean(WIS) / (len(quantiles) // 2 + 0.5)

            
            # save and format output
            if loc == 'US':
                loc_conv = loc
            elif int(loc) <10:
                loc_conv = loc[1]
            else:
                loc_conv = loc  

            # save into dataframe
            newrow = pd.DataFrame({'Model':model , 'Label': 'Scenario '+ scenario, 'location':loc_conv, 'WIS':WIS, 
                                'target':target}, index=[0])

            wisdf_traj = pd.concat([wisdf_traj, newrow])


wisdf_traj = wisdf_traj.reset_index()
wisdf_traj = wisdf_traj.drop(columns=['index'])   


In [83]:
wisdf_traj

,Model,Label,location,WIS,target
0,MOBS_NEU-GLEAM_FLU,Scenario A,1,45.891879,hosp
1,MOBS_NEU-GLEAM_FLU,Scenario B,1,37.968114,hosp
2,MOBS_NEU-GLEAM_FLU,Scenario C,1,44.269688,hosp
3,MOBS_NEU-GLEAM_FLU,Scenario D,1,36.551048,hosp
4,MOBS_NEU-GLEAM_FLU,Scenario E,1,46.840356,hosp
...,...,...,...,...,...
925,UT-ImmunoSEIRS,Scenario B,US,2728.481059,hosp
926,UT-ImmunoSEIRS,Scenario C,US,2913.826165,hosp
927,UT-ImmunoSEIRS,Scenario D,US,2734.306769,hosp
928,UT-ImmunoSEIRS,Scenario E,US,2572.279479,hosp
